In [299]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [312]:
# Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ap20/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
INFO:WDM:Driver [/Users/ap20/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


In [313]:
# Batting Stats
#
#

In [317]:
# Batting stats URLs
batting_url_list = ['https://www.espn.com/mlb/stats/player/_/view/batting/season/2021/seasontype/2/table/batting/sort/homeRuns/dir/desc',
                    'https://www.espn.com/mlb/stats/player/_/view/batting/season/2020/seasontype/2/table/batting/sort/homeRuns/dir/desc',
                    'https://www.espn.com/mlb/stats/player/_/view/batting/season/2019/seasontype/2/table/batting/sort/homeRuns/dir/desc']

In [315]:
# Splinter scrape for Abbreviation

def abbreviation(url):
  browser.visit(url)
  
  abrs = []

  for i in range(1,51):    
        
    abr = browser.find_by_xpath('//*[@id="fittPageContainer"]/div[3]/div/div/section/div/div[4]/div[1]/div/table/tbody/tr[' + str(i) + ']/td[2]/div/span').text

    abrs.append(abr)
    
  return abrs


In [56]:
# BeautifulSoup scrape for Abbreviation

#def abbreviation(url):

#        html = browser.html
#        soup = BeautifulSoup(html, 'html.parser')
    
#        results = soup.find_all('tr', class_ = 'Table__TR Table__TR--sm Table__even')

#        abrs = []

#        for query in results:
#            try:        
#                abr = query.span.text    
#                abrs.append(abr)
        
#            except AttributeError as e:
#                print(e)

In [318]:
# Abbreviation column variables
abr_2021 = abbreviation(batting_url_list[0])
abr_2020 = abbreviation(batting_url_list[1])
abr_2019 = abbreviation(batting_url_list[2])

In [319]:
# Splinter scrape for Name

def name(url):
  browser.visit(url)
  
  full_name = []

  for i in range(1,51):    
        
    name = browser.find_by_xpath('//*[@id="fittPageContainer"]/div[3]/div/div/section/div/div[4]/div[1]/div/table/tbody/tr[' + str(i) +']/td[2]/div/a').text

    full_name.append(name)
    
  return full_name

In [71]:
# BeautifulSoup scrape for Name

#def name(url):

#    html = browser.html
#    soup = BeautifulSoup(html, 'html.parser')
    
#    results = soup.find_all('tr', class_ = 'Table__TR Table__TR--sm Table__even')

#    names = []

#    for query in results:        

#        try:        
#            full_name = query.a.text    
#            names.append(full_name)
        
#        except AttributeError as e:
#            print(e)  
    
#    return names

In [321]:
# Name column variables
names_2021 = name(batting_url_list[0])
names_2020 = name(batting_url_list[1])
names_2019 = name(batting_url_list[2])

In [322]:
# Function for scraping desired batting data

def batting(url):
    tables = pd.read_html(url)

    t1 = tables[0]
    t2 = tables[1]

    batting_df = t1.join(t2)

    clean_batting_df = batting_df.drop(['OBP','SLG','OPS','WAR'], axis= 1)

    clean_batting_df.rename(columns = {'Name':'Names','GP':'Games Played', 'AB':'At Bats','R': 'Runs', 'H':'Hits', 'HR':'Home Runs',
                                        'RBI':'Runs Batted In','TB':'Total Bases', 'BB':'Walks', 'K':'Strikeouts', 'SB':'Stolen Bases'}, inplace = True)

    return clean_batting_df


In [323]:
# $READY$

# 2021 Batting Table
batting_2021_df = batting(batting_url_list[0])

# Insert new columns
batting_2021_df.insert(1,'Name',names_2021)
batting_2021_df.insert(2,'Team',abr_2021)

# Drop Teams column
batting_2021_df.drop(['Names'], axis=1, inplace=True)

# Export DF to CSV
batting_2021_df.to_csv('csvs/batting_2021.csv', index=False)

In [326]:
# $READY$

# 2020 Batting Table
batting_2020_df = batting(batting_url_list[1])

# Insert new columns
batting_2020_df.insert(1,'Name',names_2020)
batting_2020_df.insert(2,'Team',abr_2020)

# Drop Teams column
batting_2020_df.drop(['Names'], axis=1, inplace=True)

# Export DF to CSV
batting_2020_df.to_csv('csvs/batting_2020.csv', index=False)

In [210]:
# $READY$

# 2019 Batting Table
batting_2019_df = batting(batting_url_list[2])

# Insert new columns
batting_2019_df.insert(1,'Name',names_2019)
batting_2019_df.insert(2,'Team',abr_2019)

# Drop Teams column
batting_2019_df.drop(['Names'], axis=1, inplace=True)

# Export DF to CSV
batting_2019_df.to_csv('csvs/batting_2019.csv', index=False)

In [135]:
# Attendance stats
#
#

In [327]:
# Attendance URLs
attendance_url_list = ['https://www.espn.com/mlb/attendance/_/year/2021',
                            'https://www.espn.com/mlb/attendance/_/year/2019']

In [328]:
# Function to return clean dataframes for attendance

def attendance(url):
    
    tables = pd.read_html(url)

    attendance_df = tables[0]

    # Rename columns
    attendance_df.rename(columns = {0 :'RK', 1 :'Teams', 2 :'Home Games', 3 :'Home Total', 4 :'Home Avg', 6 :'Road Games', 7 :'Road Avg',
                                    9 :'Overall Games', 10 :'Overall Avg'}, inplace = True)
    # Drop by Index
    attendance_df.drop([0,1], inplace=True)

    # Reset Index
    attendance_df.reset_index(inplace=True)

    # Drop by column
    attendance_df.drop(['index',5, 8, 11], axis=1, inplace=True)

    # Created list to replace Teams column
    team = ['Arizona Diamondbacks', 'Atlanta Braves','Baltimore Orioles','Boston Red Sox','Chicago Cubs','Chicago White Sox',
            'Cincinnati Reds','Cleveland Guardians','Colorado Rockies','Detroit Tigers','Houston Astros','Kansas City Royals',
                'Los Angeles Angels','Los Angeles Dodgers','Miami Marlins','Milwaukee Brewers','Minnesota Twins','New York Mets',
                    'New York Yankees','Oakland Athletics','Philadelphia Phillies','Pittsburgh Pirates','San Diego Padres','San Francisco Giants',
                        'Seattle Mariners','St. Louis Cardinals','Tampa Bay Rays','Texas Rangers','Toronto Blue Jays','Washington Nationals']
    abr = ['ARI','ATl','BAL','BOS','CHC','CWS','CIN','CLE','COL','DET','HOU','KC','LAA','LA','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SD','SF',
            'SEA','STL','TB','TEX','TOR','WAS']

    # Place alphabetical
    attendance_df.sort_values(by=['Teams'], inplace=True)

    # Insert new columns
    attendance_df.insert(1,'Team',team)
    attendance_df.insert(2,'ABR',abr)

    # Drop Teams column
    attendance_df.drop(['Teams'], axis=1, inplace=True)

    # Convert column RK to INT
    complete_attendace_df = attendance_df.astype({"RK": int},  errors='raise') 

    # Resorts dataframe back to normal based on rank
    complete_attendace_df.sort_values(by=['RK'], inplace=True)

    # Convert necessary columns to INT
    complete_attendace_df[['Home Total', 'Home Avg','Road Avg','Overall Avg']] = complete_attendace_df[['Home Total', 'Home Avg','Road Avg','Overall Avg']].astype(int)

# Had to marker out the formatting due to conflict with exporting to csv
#
#    complete_attendace_df = complete_attendace_df.style.format({'Home Total':'{:,.0f}',
#                                                    'Home Avg':'{:,.0f}',
#                                                        'Road Avg':'{:,.0f}',
#                                                            'Overall Avg':'{:,.0f}'})

    return complete_attendace_df

In [329]:
# COMPLETE ATTENDANCE CHARTS
attendance_2021_df = attendance('https://www.espn.com/mlb/attendance/_/year/2021')
attendance_2019_df = attendance('https://www.espn.com/mlb/attendance/_/year/2019')

In [220]:
# Export DF to CSV Attendance
attendance_2021_df.to_csv('csvs/attendance_2021.csv', index=False)
attendance_2019_df.to_csv('csvs/attendance_2019.csv', index=False)

In [331]:
browser.quit()

In [ ]:
# Team Payroll
#
#

In [332]:
# Payroll URL
payroll_url = 'http://www.stevetheump.com/Payrolls.htm'

In [333]:
# Payroll DFs for each year
tables = pd.read_html(payroll_url)

payroll_2021_df = tables[2]
payroll_2020_df = tables[3]
payroll_2019_df = tables[4]

In [334]:
# Payroll 2021 clean up

# Rename columns
payroll_2021_df.rename(columns = {0 :'RK', 1 :'Team', 2 :'Roster', 3 :'Active', 4 :'Injured Reserve'}, inplace = True)

# Drop by Index
payroll_2021_df.drop([0,17], inplace=True)

# Reset Index
payroll_2021_df.reset_index(inplace=True)

# Drop by column
payroll_2021_df.drop(['index',5, 6, 7], axis=1, inplace=True)

# Specified columns to change
cols = ['Active','Injured Reserve']

# Removed commas and $ signs from specified columns to convert to INT
payroll_2021_df[cols] = payroll_2021_df[cols].replace({'\$': '', ',': ''}, regex=True)

# Replacing the - in columns with NaN values then replacing NaN values to 0
payroll_2021_df = payroll_2021_df.replace('-',np.NaN)
payroll_2021_df = payroll_2021_df.fillna(0)

# Covert desired colums to INT tobe able to make 'Total' column
payroll_2021_df[['Active', 'Injured Reserve']] = payroll_2021_df[['Active', 'Injured Reserve']].astype(int)

# 'Active' + 'Injured Reserve' = 'Total'
payroll_2021_df['Total'] = payroll_2021_df['Active'] + payroll_2021_df['Injured Reserve']


# Format
#payroll_2021_df = payroll_2021_df.style.format({'Active':'${:,.0f}',
#                                                    'Injured Reserve':'${:,.0f}',
#                                                        'Total':'${:,.0f}'})

In [228]:
# Payroll 2021 DF export to CSV
payroll_2021_df.to_csv('csvs/payroll_2021.csv', index=False)

In [336]:
# Payroll 2020 clean up

# Rename columns
payroll_2020_df.rename(columns = {0 :'RK', 1 :'Team', 2 :'Roster', 3 :'Active', 4 :'Injured Reserve'}, inplace = True)

# Drop by Index
payroll_2020_df.drop([0,17], inplace=True)

# Reset Index
payroll_2020_df.reset_index(inplace=True)

# Drop by column
payroll_2020_df.drop(['index',5, 6], axis=1, inplace=True)

# Specified columns to change
cols = ['Active','Injured Reserve']

# Removed commas and $ signs from specified columns to convert to INT
payroll_2020_df[cols] = payroll_2020_df[cols].replace({'\$': '', ',': ''}, regex=True)

# Replacing the - in columns with NaN values then replacing NaN values to 0
payroll_2020_df = payroll_2020_df.replace('-',np.NaN)
payroll_2020_df = payroll_2020_df.fillna(0)

# Covert desired colums to INT tobe able to make 'Total' column
payroll_2020_df[['Active', 'Injured Reserve']] = payroll_2020_df[['Active', 'Injured Reserve']].astype(int)

# 'Active' + 'Injured Reserve' = 'Total'
payroll_2020_df['Total'] = payroll_2020_df['Active'] + payroll_2020_df['Injured Reserve']

# Format
#payroll_2020_df = payroll_2020_df.style.format({'Active':'${:,.0f}',
#                                                    'Injured Reserve':'${:,.0f}',
#                                                        'Total':'${:,.0f}'})

In [230]:
# Payroll 2020 DF export to CSV
payroll_2020_df.to_csv('csvs/payroll_2020.csv', index=False)

In [337]:
# Payroll 2019 clean up

# Rename columns
payroll_2019_df.rename(columns = {0 :'RK', 1 :'Team', 2 :'Roster', 3 :'Active', 4 :'Injured Reserve'}, inplace = True)

# Drop by Index
payroll_2019_df.drop([0,14], inplace=True)

# Reset Index
payroll_2019_df.reset_index(inplace=True)

# Drop by column
payroll_2019_df.drop(['index'], axis=1, inplace=True)

# Specified columns to change
cols = ['Active','Injured Reserve']

# Removed commas and $ signs from specified columns to convert to INT
payroll_2019_df[cols] = payroll_2019_df[cols].replace({'\$': '', ',': ''}, regex=True)

# Replacing the - in columns with NaN values then replacing NaN values to 0
payroll_2019_df = payroll_2019_df.replace('-',np.NaN)
payroll_2019_df = payroll_2019_df.fillna(0)

# Covert desired colums to INT tobe able to make 'Total' column
payroll_2019_df[['Active', 'Injured Reserve']] = payroll_2019_df[['Active', 'Injured Reserve']].astype(int)

# 'Active' + 'Injured Reserve' = 'Total'
payroll_2019_df['Total'] = payroll_2019_df['Active'] + payroll_2019_df['Injured Reserve']

# Format
#payroll_2019_df = payroll_2019_df.style.format({'Active':'${:,.0f}',
#                                                    'Injured Reserve':'${:,.0f}',
#                                                        'Total':'${:,.0f}'})

In [232]:
# Payroll 2021 DF export to CSV
payroll_2019_df.to_csv('csvs/payroll_2019.csv', index=False)